# Power output for REZ for different energy scenarios

In [1]:
from dask.distributed import Client,LocalCluster
from dask_jobqueue import PBSCluster

In [144]:
client.close()
cluster.close()

In [145]:
# One node on Gadi has 48 cores - try and use up a full node before going to multiple nodes (jobs)

walltime = "01:10:00"
cores = 48
memory = str(4 * cores) + "GB"

cluster = PBSCluster(walltime=str(walltime), cores=cores, memory=str(memory), processes=cores,
                     job_extra_directives=["-q normal",
                                           "-P w42",
                                           "-l ncpus="+str(cores),
                                           "-l mem="+str(memory),
                                           "-l storage=gdata/w42+gdata/rt52"],
                     local_directory="$TMPDIR",
                     job_directives_skip=["select"])

In [146]:
cluster.scale(jobs=3)
client = Client(cluster)

In [147]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.121.9:42695,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [5]:
import xarray as xr
import pandas as pd

In [6]:
%cd /g/data/w42/dr6273/work/power_models
import functions as fn

/g/data/w42/dr6273/work/power_models


In [7]:
%load_ext autoreload
%autoreload 2

# Load REZ data

#### REZ mask

In [9]:
mask = xr.open_dataset('/g/data/w42/dr6273/work/projects/Aus_energy/data/rez_2024_mask_era5_grid.nc').REZ

#### REZ generation

In [39]:
_gen = pd.read_csv("/g/data/w42/dr6273/work/data/REZ/2024/REZ_potential.csv", index_col=0)

In [40]:
# Tidy column names
_gen.columns = [i.replace("90", "9-") for i in _gen.columns]
_gen.columns = [i.replace(" ", "_") for i in _gen.columns]

In [42]:
# Sum scenarios with existing capacity
gen = _gen.copy()
for col in _gen.columns[2:]:
    if col[:3] == "Exi":
        pass
    else:
        gen_type = col.split("_")[-2]
        gen[col] = _gen[col] + _gen["Existing_" + gen_type]

In [43]:
gen.head()

,Solar_renewable_potential_(MW),Wind_renewable_potential_(MW),Existing_solar,Progressive_solar_2029-30,Progressive_solar_2039-40,Progressive_solar_2049-50,Step_change_solar_2029-30,Step_change_solar_2039-40,Step_change_solar_2049-50,Green_energy_solar_2029-30,...,Existing_wind,Progressive_wind_2029-30,Progressive_wind_2039-40,Progressive_wind_2049-50,Step_change_wind_2029-30,Step_change_wind_2039-40,Step_change_wind_2049-50,Green_energy_wind_2029-30,Green_energy_wind_2039-40,Green_energy_wind_2049-50
REZ,,,,,,,,,,,,,,,,,,,,,
N1,6385,0,166,216,216,266,166,216,266,216,...,0,0,0,0,0,0,0,0,0,-300
N2,2950,7400,855,905,1355,3855,855,3255,6255,3155,...,442,3492,4042,7842,3442,7842,7842,4992,9142,9592
N3,6850,3000,1497,3197,3447,7597,2947,5897,8347,4447,...,673,4373,4373,4873,6123,6123,8473,7373,7473,8173
N4,8000,5100,53,253,253,403,203,203,453,253,...,198,298,298,298,298,298,348,348,348,348
N5,2256,3900,1122,1822,1822,3372,2272,2272,3372,1922,...,0,450,450,450,600,600,1000,1000,1000,1000


# Compute generation

In [67]:
def get_scaled_capacity_mask(mask_da, generation):
    """
    Return DataArray with mask of capacity divided evenly across grid cells
    
    mask_da: array of REZ mask
    generation: pandas Series of capacity values for each region (index).
    """
    da_list = []
    for r in generation.index:
        capacity = generation.loc[r]
        n_cells = mask_da.sel(region=r).sum().values
        
        da = mask_da.sel(region=r).where(
            mask_da.sel(region=r) == 0,
            capacity / n_cells
        ).expand_dims({"REZ": [r]})
        
        da_list.append(da)
        
    return xr.concat(da_list, dim="REZ")

In [72]:
def calc_generation(capacity_factor, generation_capacity):
    """
    Return DataArray with time series of power for each REZ
    
    capacity_factor: DataArray of capacity factors
    generation_capacity: DataArray of each grid cells generation capacity
    """
    da_list = []
    for r in generation_capacity.REZ.values:
        r_mask = generation_capacity.sel(REZ=r).where(
            generation_capacity.sel(REZ=r) > 0, drop=True
        )
        da = capacity_factor * r_mask
        da = da.sum(["lat", "lon"])
        da = da.expand_dims({"REZ": [r]})

        da_list.append(da)

    return xr.concat(da_list, dim="REZ")

In [136]:
def calc_all_years_generation(years, mask_da, generation, method="van_der_Wiel"):
    """
    Compute power time series for each REZ and each year.
    
    years: range
    facilities: dict of facilities data in dataframe
    method: str, which method of capacity factors was used
    """
    def _get_gen_type(gen):
        g_split = gen.name.split("_")
        if g_split[0] == "Solar":
            gt = "solar"
        elif g_split[0] == "Wind":
            gt = "wind"
        elif g_split[0] == "Existing":
            gt = g_split[1]
        else:
            gt = g_split[-2]
        return gt
    
    gen_type = _get_gen_type(generation)
    gen_capacity = get_scaled_capacity_mask(mask_da, generation)
    
    da_list = []
    for year in years:
        cf = fn.load_hourly_cf(year, gen_type, method=method, chunks={"lat": -1, "lon": -1, "time": 2500}).capacity_factor
        p = calc_generation(cf, gen_capacity)
        da_list.append(p)
    REZ_power = xr.concat(da_list, dim="time")
    REZ_power = REZ_power.chunk({"REZ": -1, "time": -1})
    
    return REZ_power

In [139]:
# cf = fn.load_hourly_cf(1940, "wind", method="van_der_Wiel", chunks={"lat": -1, "lon": -1, "time": 2500}).capacity_factor

In [96]:
fp = "/g/data/w42/dr6273/work/projects/Aus_energy/production_metrics/"

In [141]:
years = range(1940, 2024)

In [143]:
len(gen.columns)

22

In [148]:
%%time
for scenario in gen.columns[7:]:
    print(scenario)
    gen_ds = calc_all_years_generation(years, mask, gen[scenario])
    gen_ds = gen_ds.to_dataset(name="power")
    
    # Tidy path to write
    if scenario.split("_")[-1] == "(MW)":
        scenario = scenario[:-5]
    scenario = scenario.lower()
    
    gen_ds.to_netcdf(
        fp + "REZ_power_" + scenario + "_van_der_Wiel" + "_era5_hourly_" + str(years[0]) + "-" + str(years[-1]) + ".nc"
    )

Step_change_solar_2039-40
Step_change_solar_2049-50
Green_energy_solar_2029-30
Green_energy_solar_2039-40
Green_energy_solar_2049-50
Existing_wind
Progressive_wind_2029-30
Progressive_wind_2039-40
Progressive_wind_2049-50
Step_change_wind_2029-30
Step_change_wind_2039-40
Step_change_wind_2049-50
Green_energy_wind_2029-30
Green_energy_wind_2039-40
Green_energy_wind_2049-50
CPU times: user 39min 11s, sys: 2min 2s, total: 41min 14s
Wall time: 41min 7s


In [158]:
gen_path = "/g/data/w42/dr6273/work/data/Open_NEM/"

In [159]:
facilities = {
    'NSW': pd.read_csv(gen_path+"edited/facilities_solar_wind_NSW_edited.csv"),
    'QLD': pd.read_csv(gen_path+"edited/facilities_solar_wind_QLD_edited.csv"),
    'SA': pd.read_csv(gen_path+"edited/facilities_solar_wind_SA_edited.csv"),
    'VIC': pd.read_csv(gen_path+"edited/facilities_solar_wind_VIC_edited.csv"),
    'TAS': pd.read_csv(gen_path+"edited/facilities_solar_wind_TAS_edited.csv"),
    'WEM': pd.read_csv(gen_path+"edited/facilities_solar_wind_WEM_edited.csv")
}

In [152]:
def get_REZ_for_state(ds, state):
    """
    Return list of REZ names for state
    
    ds: dataset with REZ dimension
    state: str, code or for state e.g. NSW, SA
    """
    return [i for i in ds.region.values if i[0] == state[0]]

In [168]:
gen.loc[get_REZ_for_state(mask, "NSW")]["Existing_wind"].sum()

1696

In [169]:
facilities["NSW"].where(facilities["NSW"]["Technology"] == "Wind")["Generator Capacity (MW)"].sum()

2568.46